In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SimpleHudiCreate") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
    .getOrCreate()


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55442)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [2]:
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import lit

data = [
    Row(id=1, name="Alice", ts=datetime.now()),
    Row(id=2, name="Bob", ts=datetime.now())
]

df = spark.createDataFrame(data).withColumn("partition", lit("2025"))


In [3]:
hudi_path = "/home/jovyan/hudi/simple_hudi_table"

df.write.format("hudi") \
    .option("hoodie.table.name", "simple_hudi_table") \
    .option("hoodie.datasource.write.recordkey.field", "id") \
    .option("hoodie.datasource.write.precombine.field", "ts") \
    .option("hoodie.datasource.write.partitionpath.field", "partition") \
    .option("hoodie.datasource.write.operation", "insert") \
    .mode("overwrite") \
    .save(hudi_path)


In [4]:
df_read = spark.read.format("hudi").load(hudi_path + "/*")
df_read.show()


+-------------------+--------------------+------------------+----------------------+--------------------+---+-----+--------------------+---------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name| id| name|                  ts|partition|
+-------------------+--------------------+------------------+----------------------+--------------------+---+-----+--------------------+---------+
|  20250712052738501|20250712052738501...|                 1|                  2025|f22f58b6-5e55-448...|  1|Alice|2025-07-12 05:27:...|     2025|
|  20250712052738501|20250712052738501...|                 2|                  2025|f22f58b6-5e55-448...|  2|  Bob|2025-07-12 05:27:...|     2025|
+-------------------+--------------------+------------------+----------------------+--------------------+---+-----+--------------------+---------+

